In [ ]:
!pip install seleniumbase 
!pip install selenium
!pip install python-docx

In [1]:
from seleniumbase import Driver
from seleniumbase import page_actions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import os
import requests
import time
import csv

In [8]:
class Leetcode:
    def __init__(self):
        self.driver = Driver(uc=True)
        self.tag_list=[]
        self.companies_list=[]
        self.cno = 0
        self.failed_list =[]
        self.top_question_no_tag_list =[] 
        self.tno = 0

    def Login(self,id,pw):
        url = "https://leetcode.com/accounts/login/"
        self.driver.get(url)
        id_field = self.driver.find_element(by=By.ID, value="id_login")
        id_field.send_keys(id)
        time.sleep(1)
        pw_field = self.driver.find_element(by=By.ID, value="id_password")
        pw_field.send_keys(pw)
        #no need check ctp
        WebDriverWait(self.driver, 5).until(EC.element_to_be_clickable((By.ID, "signin_btn"))).click()
        time.sleep(5)

    def SetTopNo(self,no):
        self.tno = no

    def GetTitleToWebText(self,text):
        text = text.lower()
        text = text.replace(' ', '-')
        text = text.replace("(", '')
        text = text.replace(")", '')
        text = text.replace(",", '-')
        text = text.replace("'", '')
        text = text.replace("%", '')
        return text
      
    def GetTopQuestionWithoutTag(self):
        url = "https://leetcode.com/problemset/"
        self.driver.get(url)
        WebDriverWait(self.driver, 5).until(EC.element_to_be_clickable((By.XPATH, "(//div[contains(text(),'Frequency')])[1]"))).click()
        time.sleep(3)
        cols = self.driver.find_elements(by = By.XPATH,value="(//div[@role='cell'])")
        i = 1
        for c in cols:
            if i > 7:
                column = i % 6
                if column == 2:
                    text = c.text
                    no = text.split('. ')[0]
                    self.top_question_no_tag_list.append(no)
                    title = text.split('. ')[1]
                    self.top_question_no_tag_list.append(title)
                elif column == 4 or column == 5:
                    self.top_question_no_tag_list.append(c.text)
                elif column == 0:
                    html_text = c.get_attribute("innerHTML")
                    target_text = html_text.rsplit("style")[1]
                    target_text = target_text.split(";")[0]
                    target_text = target_text.replace('"','')
                    target_text = target_text.replace('=','')
                    target_text = target_text.replace('width','')
                    target_text = target_text.replace(':','')
                    target_text = target_text.replace(' ','')
                    self.top_question_no_tag_list.append(target_text)    
            i +=1
        WebDriverWait(self.driver,10).until(EC.element_to_be_clickable((By.XPATH, "(//button[normalize-space()='2'])[1]"))).click()
        i = 1
        time.sleep(3)
        cols = self.driver.find_elements(by = By.XPATH,value="(//div[@role='cell'])")
        for c in cols:
            column = i % 6
            if column == 2:
                text = c.text
                no = text.split('. ')[0]
                self.top_question_no_tag_list.append(no)
                title = text.split('. ')[1]
                self.top_question_no_tag_list.append(title)
            elif column == 4 or column == 5:
                self.top_question_no_tag_list.append(c.text)
            elif column == 0:
                html_text = c.get_attribute("innerHTML")
                target_text = html_text.rsplit("style")[1]
                target_text = target_text.split(";")[0]
                target_text = target_text.replace('"','')
                target_text = target_text.replace('=','')
                target_text = target_text.replace('width','')
                target_text = target_text.replace(':','')
                target_text = target_text.replace(' ','')
                self.top_question_no_tag_list.append(target_text)
            i +=1
        time.sleep(10)
       

    def GetTopQuestionTag(self,title):
        tag = ""
        url = "https://leetcode.com/problems/" + self.GetTitleToWebText(title) +"/description/"
        self.driver.get(url)
        try:
            WebDriverWait(self.driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div[class='flex gap-1'] div:nth-child(2)"))).click()
            tag_elements_parent = self.driver.find_element(by = By.CSS_SELECTOR, value=".mt-2.flex.flex-wrap.gap-1.pl-7")
            time.sleep(1)
            tag_elements = tag_elements_parent.find_elements(by = By.TAG_NAME, value="a")
            for e in tag_elements:
                text = "[" + e.text + "]"
                tag += text
            time.sleep(1)
        except:
            print("no tag")
        return tag

    def GetTopQuestion(self):
        self.GetTopQuestionWithoutTag()
        dest_folder = 'Companies Leetcode'
        if not os.path.exists(dest_folder):
            os.makedirs(dest_folder)
        file_path = "top question.csv"
        with open(file_path, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["No", "Title","Tags","Acceptance","Difficulty","Frequency"])
            i = 0
            j = 0
            while j < 100:
                no = self.top_question_no_tag_list[i]
                title = self.top_question_no_tag_list[i+1]
                tags = self.GetTopQuestionTag(title)
                acceptance = self.top_question_no_tag_list[i+2]
                difficulty = self.top_question_no_tag_list[i+3]
                frequency = self.top_question_no_tag_list[i+4]
                writer.writerow([no, title ,tags,acceptance,difficulty,frequency])
                i += 5
                j += 1
            file.close()
               


    def GetAllTagAndCompaniesList(self):
        url = "https://leetcode.com/problemset/"
        self.driver.get(url)
        WebDriverWait(self.driver, 5).until(EC.element_to_be_clickable((By.XPATH, "(//button[@id='headlessui-popover-button-:r5:'])[1]"))).click()
        WebDriverWait(self.driver, 2).until(EC.element_to_be_clickable((By.XPATH, "(//div[@class='text-blue dark:text-dark-blue m-1 flex cursor-pointer items-center px-1'][normalize-space()='Expand'])[1]"))).click()
        tag_elements_parent = self.driver.find_element(by = By.XPATH, value="(//div[@class='-m-1 mt-1 flex max-h-[400px] flex-wrap overflow-auto py-4'])[1]")
        tag_elements = tag_elements_parent.find_elements(by = By.TAG_NAME, value="span")
        with open('tag.txt', 'w') as f:
            for e in tag_elements:
                self.tag_list.append(e.text)
                f.write(e.text)
                f.write('\n')
        f.close()
        WebDriverWait(self.driver, 2).until(EC.element_to_be_clickable((By.XPATH, "(//div[@class='text-sm leading-5 font-normal text-label-2 dark:text-dark-label-2'])[1]"))).click()
        WebDriverWait(self.driver, 2).until(EC.element_to_be_clickable((By.XPATH, "(//div[contains(text(),'Expand')])[1]"))).click()
        companies_elements_parent = self.driver.find_element(by = By.XPATH, value="(//div[@class='-m-1 mt-1 flex max-h-[400px] flex-wrap overflow-auto py-4'])[2]")
        companies_elements = companies_elements_parent.find_elements(by = By.TAG_NAME, value="span")
        with open('companies.txt', 'w') as f:
            for e in companies_elements:
                self.companies_list.append(e.text)
                f.write(e.text)
                f.write('\n')
        f.close()
        time.sleep(2)
        self.GetCompaniesQuestion()

    def TextToWebText(self,text):
        if text == "Quip (Salesforce)":
            return "quip"

        if text == "Virtu Financial":
            return "virtu"
            
        text = text.lower()
        text = text.replace(' ', '-')
        text = text.replace('.', '')
        return text

    def GetCompaniesQuestion(self):
        dest_folder = 'Companies Leetcode'
        if not os.path.exists(dest_folder):
            os.makedirs(dest_folder)
        
        if self.cno < len(self.companies_list):
            company_name = self.companies_list[self.ccno]
            file_name = company_name + ".csv"
            file_path = os.path.join(dest_folder, file_name)
            url = "https://leetcode.com/company/" + self.TextToWebText(company_name) + "/"
            self.driver.get(url)
            WebDriverWait(self.driver, 2).until(EC.element_to_be_clickable((By.XPATH, "(//input[@type='checkbox'])[1]"))).click()
            WebDriverWait(self.driver, 2).until(EC.element_to_be_clickable((By.XPATH, "(//th[normalize-space()='Frequency'])[1]"))).click()
            WebDriverWait(self.driver, 1).until(EC.element_to_be_clickable((By.XPATH, "(//th[normalize-space()='Frequency'])[1]"))).click()
            table = self.driver.find_element(by = By.CSS_SELECTOR, value=".reactable-data")
            rows = table.find_elements(by = By.TAG_NAME, value="tr")
            with open(file_path, 'w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(["No", "Title","Tags","Acceptance","Difficulty","Frequency"])

                error =False
                for r in rows:
                    cols = r.find_elements(by = By.TAG_NAME, value="td")
                    if len(cols) == 7:
                        no = cols[1].text
                        title = cols[2].text
                        tags_list = cols[3].find_elements(by = By.TAG_NAME, value="a")
                        tags = ""
                        for t in tags_list:
                            tags = tags + "[" + t.text + "]"
                        acceptance = cols[4].text
                        difficulty = cols[5].text
                        frequency = cols[6].get_attribute("value")
                        writer.writerow([no, title ,tags,acceptance,difficulty,frequency])
                    else:
                        error = True
                file.close()
            if error == True:
                self.failed_list.append(company_name)
                print("Data mining failed.",company_name)
            else:
                print("Done data mining",company_name)
                
            time.sleep(2)
            self.cno += 1
            self.GetCompaniesQuestion()
        else:
            self.driver.quit()
                    


In [9]:
test = Leetcode()


In [10]:
test.GetTopQuestion()

no tag
no tag
no tag
no tag
no tag
no tag


NameError: name 'close' is not defined

In [11]:
import csv
import os
import pandas as pd

In [15]:
class LeetcodeDataFrameConvert:
    def __init__(self):
        self.companies_list=[]
        self.all_tags_list=[]

    def InitTagsList(self):
        cf = open("tag.txt", "r")
        for c in cf:
            c = c.replace("\n", "")
            self.all_tags_list.append(c)
        cf.close

    
    def InitCompaniesList(self):
        cf = open("companies.txt", "r")
        for c in cf:
            c = c.replace("\n", "")
            self.companies_list.append(c)
        cf.close

    
    def ConvertTopQuestionToDataframe(self):
        dest_folder = 'Companies Leetcode DataFrame'
        if not os.path.exists(dest_folder):
            os.makedirs(dest_folder)
        c =  'top question'
        read_filename = c
        read_filename +='.csv'
        read_file_path = os.path.join('Companies Leetcode', read_filename)
        df = pd.read_csv(read_file_path)
        #difficulty = pd.get_dummies(df['Difficulty']).astype(int)
        #df = df.drop(columns=['Difficulty'])
        #df = pd.concat([df,difficulty], axis= 1)
        tags_header_list = []
        for index, row in df.iterrows():
            tags = str(row['Tags'])
            tags = tags.replace('[', '')
            tags_list = tags.split(']')
            if len(tags_list) != 0 and tags_list[len(tags_list)-1]=="":
                tags_list= tags_list[:-1]
            if len(tags_list) == 0 or tags_list[0] =="nan":
                continue
            for tag in tags_list:
                if tag not in tags_header_list:
                    tags_header_list.append(tag)
        for tag in tags_header_list:
            df[tag] = 0
        df2 = df
        for index, row in df2.iterrows():
            tags = str(row['Tags'])
            tags = tags.replace('[', '')
            tags_list = tags.split(']')
            if len(tags_list) != 0 and tags_list[len(tags_list)-1]=="":
                tags_list= tags_list[:-1]
            if len(tags_list) == 0 or tags_list[0] =="nan":
                continue
            for tag in tags_list:
                df.loc[index, tag] = 1
        df = df.drop(columns=['Tags'])
        file_name = c + ".csv"
        write_file_path = os.path.join(dest_folder, file_name)
        df.to_csv(write_file_path, encoding='utf-8', index=False)
        print(c,"file successfully converted to data frame format.")

    def ConvertTopQuestionToDataframeAllTag(self):
        self.InitTagsList()
        dest_folder = 'Companies Leetcode DataFrame All Tag'
        if not os.path.exists(dest_folder):
            os.makedirs(dest_folder)
                       
        c =  'top question'
        read_filename = c
        read_filename +='.csv'
        read_file_path = os.path.join('Companies Leetcode', read_filename)
        df = pd.read_csv(read_file_path)
        tags_header_list = self.all_tags_list
        for tag in tags_header_list:
            df[tag] = 0
        df2 = df
        for index, row in df2.iterrows():
            tags = str(row['Tags'])
            tags = tags.replace('[', '')
            tags_list = tags.split(']')
            if len(tags_list) != 0 and tags_list[len(tags_list)-1]=="":
                tags_list= tags_list[:-1]
            if len(tags_list) == 0 or tags_list[0] =="nan":
                continue
        for tag in tags_list:
            df.loc[index, tag] = 1
        df = df.drop(columns=['Tags'])
        file_name = c + ".csv"
        write_file_path = os.path.join(dest_folder, file_name)
        df.to_csv(write_file_path, encoding='utf-8', index=False)
        print(c,"file successfully converted to data frame format.")

    def ConvertToDataframe(self):
        dest_folder = 'Companies Leetcode DataFrame'
        if not os.path.exists(dest_folder):
            os.makedirs(dest_folder)
                       
        for c in self.companies_list:
            read_filename = c
            read_filename +='.csv'
            read_file_path = os.path.join('Companies Leetcode', read_filename)
            df = pd.read_csv(read_file_path)
            #difficulty = pd.get_dummies(df['Difficulty']).astype(int)
            #df = df.drop(columns=['Difficulty'])
            #df = pd.concat([df,difficulty], axis= 1)
            tags_header_list = []
            for index, row in df.iterrows():
                tags = str(row['Tags'])
                tags = tags.replace('[', '')
                tags_list = tags.split(']')
                if len(tags_list) != 0 and tags_list[len(tags_list)-1]=="":
                    tags_list= tags_list[:-1]
                if len(tags_list) == 0 or tags_list[0] =="nan":
                    continue
                for tag in tags_list:
                    if tag not in tags_header_list:
                        tags_header_list.append(tag)
            for tag in tags_header_list:
                df[tag] = 0
            df2 = df
            for index, row in df2.iterrows():
                tags = str(row['Tags'])
                tags = tags.replace('[', '')
                tags_list = tags.split(']')
                if len(tags_list) != 0 and tags_list[len(tags_list)-1]=="":
                    tags_list= tags_list[:-1]
                if len(tags_list) == 0 or tags_list[0] =="nan":
                    continue
                for tag in tags_list:
                    df.loc[index, tag] = 1
            df = df.drop(columns=['Tags'])
            file_name = c + ".csv"
            write_file_path = os.path.join(dest_folder, file_name)
            df.to_csv(write_file_path, encoding='utf-8', index=False)
            print(c,"file successfully converted to data frame format.")

    def ConvertToDataframeAllTag(self):
        self.InitTagsList()
        dest_folder = 'Companies Leetcode DataFrame All Tag'
        if not os.path.exists(dest_folder):
            os.makedirs(dest_folder)
                       
        for c in self.companies_list:
            read_filename = c
            read_filename +='.csv'
            read_file_path = os.path.join('Companies Leetcode', read_filename)
            df = pd.read_csv(read_file_path)
            tags_header_list = self.all_tags_list
            for tag in tags_header_list:
                df[tag] = 0
            df2 = df
            for index, row in df2.iterrows():
                tags = str(row['Tags'])
                tags = tags.replace('[', '')
                tags_list = tags.split(']')
                if len(tags_list) != 0 and tags_list[len(tags_list)-1]=="":
                    tags_list= tags_list[:-1]
                if len(tags_list) == 0 or tags_list[0] =="nan":
                    continue
                for tag in tags_list:
                    df.loc[index, tag] = 1
            df = df.drop(columns=['Tags'])
            file_name = c + ".csv"
            write_file_path = os.path.join(dest_folder, file_name)
            df.to_csv(write_file_path, encoding='utf-8', index=False)
            print(c,"file successfully converted to data frame format.")

        
    

In [16]:
test_df = LeetcodeDataFrameConvert()
#test_df.ConvertTopQuestionToDataframe()
test_df.ConvertTopQuestionToDataframeAllTag()
#test_df.InitCompaniesList()
#test_df.ConvertToDataframe()
#test_df.ConvertToDataframeAllTag()

top question file successfully converted to data frame format.
